# Список must know функций на данном этапе:

1) Объявлять, обучать и делать предсказание с помощью модели (fit, predict, predict_proba)

2) Разбивать выборку на трейн и тест: train_test_split

3) Проверять качество модели на кросс-валидации: cross_val_score

4) Подбирать гиперпараметры модели по кросс-валидации: GridSearchCV

5) Уметь делать пайплайны (Pipeline) + подбирать гиперпараметры моделей из пайплайна по кросс-валидации (GridSearchCV).

# Больше практики.

In [ ]:
from __future__ import division, print_function
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

### Попытаемся ответить на вопрос, что делать, если качество модели нас не устраивает?

Есть несколько путей решения:

a) добавить признаков

б) добавить данных

в) усложнить/упростить/поменять модель

Изначально не очевидно, какой (или какие) подходы позволят улучшить качество модели.

Посмотрим на пример. Будем работать с данными по оттоку клиентов телеком-оператора.

In [ ]:
data = pd.read_csv('telecom_churn.csv').drop('State', axis=1)
data.head()

Преобразуйте колонку International plan таким образом, чтобы вместо Yes в ней стояла 1, а вместо No - 0.

Аналогичным образом преобразуйте колонку Voice mail plan.

Затем преобразуйте значения колонки Churn (целевой столбец) в 1 (если True) и 0 (если False).

In [ ]:
#your code here

In [ ]:
data.head()

Создайте матрицу объект-признак из таблицы data и целевой вектор (data['Churn']), затем выкиньте из матрицы вектор 'Churn'.

In [ ]:
#your code here

Перед применением линейной модели **необходимо масштабировать признаки**. Создадим пайплайн, в котором сначала происходит масштабирование, а затем применяется модель. В данном случае будем использовать логистическую регрессию.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

logit_pipe = Pipeline([('scaler', StandardScaler()), \
                       ('logit', LogisticRegression(class_weight='balanced'))])

Обучите модель (logit_pipe) по кросс-валидации и выведите на экран roc-auc (используйте функцию cross_val_score).

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

#your code here

Как правило, подбор гиперпараметров алгоритма улучшает его качество. Подберём значение параметра регуляризации C в логистической регрессии по кросс-валидации, используя GridSearchCV.

In [ ]:
model = LogisticRegression()

param = {'C': [1, 10, 100]}

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = {'logit__C': np.logspace(-2, 0, 20)}

grid_logit = GridSearchCV(logit_pipe, param_grid, cv=3, n_jobs=-1)
grid_logit.fit(X, y)

In [ ]:
grid_logit.best_params_, grid_logit.best_score_

$1, x_1, x_2$

Добавим квадратичные признаки: $1, x_1, x_2, x_1^2, x_2^2, x_1x_2$

Добавим признаки степени три: $1, x_1, x_2, x_1^2, x_2^2, x_1x_2, x_1^3, x_2^3, x_1^2x_2, x_1x_2^2$

Попробуем улучшить модель путём добавления новых признаков. **Добавьте в модель все квадратичные признаки, используя функцию PolynomialFeatures**.

Тогда происходит три действия: масштабирование, добавление признаков и применение модели. Объедините их в один пайплайн.

In [ ]:
#your code here

Подберите значение параметра C по кросс-валидации

In [ ]:
%%time

param_grid = {'logit__C': np.logspace(-2, 0, 20)}

#your code here

In [ ]:
grid_logit2.best_params_, grid_logit2.best_score_

### Комментарии

1. Ошибка на обучающей выборке сама по себе ничего не говорит о качестве модели

2. Кросс-валидационная ошибка показывает, насколько хорошо модель подстраивается под данные (имеющийся тренд в данных), сохраняя при этом способность обобщения на новые данные

3. В данной задаче качество модели улучшилось при добавлении квадратичных признаков.

### Применим метод K ближайших соседей для решения задачи.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier()

cross_val_score(knn_model, X, y, cv=3, scoring='roc_auc').mean()

Подберите число соседей n_neighbors в методе с помощью GridSearchCV. Какое число соседей получилось оптимальным? Какое качество показал алгоритм на кросс-валидации?

In [ ]:
%%time

#your code here

In [ ]:
grid_knn.best_params_, grid_knn.best_score_

Добавьте квадратичных признаков в модель и создайте пайплайн из добавления признаков и применения метода KNN. Найдите оптимальное количество соседей по кросс-валидации. Какое количество соседей получилось в этой модели? Какое качество показал алгоритм на кросс-валидации?

In [ ]:
#your code here

In [ ]:
grid_knn2.best_params_, grid_knn2.best_score_

# Задание.

Примените метод опорных векторов для решения данной задачи. 

Используйте пайплайн, состоящий из масштабирования, извлечения квадратичных признаков и применения SVM.

Вычислите качество алгоритма на кросс-валидации.

In [ ]:
from sklearn.svm import SVC

#your code here

Подберите значение параметра регуляризации C по кросс-валидации и нарисуйте валидационную кривую, отражающую качество в зависимости от C. Сделайте выводы.

In [ ]:
#your code here

# Задание.

Примените наивный байесовский классификатор, попробуйте несколько классификаторов отсюда (https://scikit-learn.org/stable/modules/naive_bayes.html) для решения данной задачи. 

Используйте пайплайн, состоящий из масштабирования, извлечения квадратичных признаков и применения SVM.

Вычислите качество алгоритма на кросс-валидации.

In [ ]:
#your code her

Посмотрите на количество 0 и 1 в целевой переменной. Является ли выборка сбалансированной?

In [ ]:
#your code here
sum(y), len(y)-sum(y)

Добавьте в обученную выше последнюю версию логистической регрессии параметр **class_weight = 'balanced'**. Посмотрите на качество на кросс-валидации.

In [ ]:
#your code here

Аналогично - добавьте в модель SVM параметр class_weight = 'balanced' и выведите на экран качество алгоритма на кросс-валидации.

In [ ]:
#your code here

## ROC-кривая

Постром roc-кривую для наилучшей модели и для наихудшей. Для этого сначала разобъем данные на train и test, на train обучим модель, а на test сделаем предсказание - будем предсказывать вероятности классов.

In [ ]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3, random_state=1)

In [ ]:
logit_pipe.fit(Xtrain, ytrain)

log_predict = logit_pipe.predict_proba(Xtest)

In [ ]:
from sklearn.metrics import roc_curve, auc

fpr, tpr, threshold = roc_curve(ytest, log_predict[:,1])
roc_auc = auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Что такое ядра в SVM?*

Посмотрим, улучшит ли качество применение SVM с ядром. Попробуйте использовать SVM с различными ядрами для данной задачи. 

Для каждого ядра подберите значение параметра C по кросс-валидации.

In [ ]:
#your code here